In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torchvision.transforms import Resize

from tqdm import tqdm
import imageio
import gc

from sklearn.model_selection import train_test_split

In [2]:
cfg = dict()
cfg['n_channels'] = 50
cfg['input_channels'] = 1
cfg['output_channels'] = 1 # 11
cfg['dropout'] = 0.2
cfg['fc_intermediate_len'] = 100 # ? 128
cfg['x_size'] = 150 # ? such order of dims
cfg['y_size'] = 300 # ? such order of dims

In [3]:
def load_pic(image_id, is_train=True):
    _ = 'train' if is_train else 'test'
    directory = 'original_data/{}/{}/{}/{}/'.format(_, image_id[0], image_id[1], image_id[2])
    return torch.Tensor(imageio.imread(directory + image_id + '.png') )

In [4]:
# only Carbon counts
class PretrainRegressionDataset(torch.utils.data.Dataset):
    def __init__(self, file, y_file, x_size=256, y_size=256):
        self.data = pd.read_csv(file)
        self.y = torch.tensor(np.load(y_file)[:,1],
                              dtype=torch.float32).unsqueeze(1) # only C
        self.y.to(device)
        self.x_size = x_size
        self.y_size = y_size
        self.resize = Resize(size=(x_size,y_size))
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        pic = load_pic(self.data['image_id'][index]).unsqueeze(0) # C=1 channel
        pic = self.resize(pic) / 255.
        y = self.y[index]
        return pic, y

In [5]:
"""
dataset = PretrainRegressionDataset('data/pretrain2/data_train_small.csv',
                                    'data/pretrain2/atom_counts_train_small.npy',
                                     x_size=cfg['x_size'], y_size=cfg['y_size'])

count = 0
for elem in dataset:
    if count > 5: break
    print(elem[0].shape)
    print(elem)
    count += 1

del dataset
"""

"\ndataset = PretrainRegressionDataset('data/pretrain2/data_train_small.csv',\n                                    'data/pretrain2/atom_counts_train_small.npy',\n                                     x_size=cfg['x_size'], y_size=cfg['y_size'])\n\ncount = 0\nfor elem in dataset:\n    if count > 5: break\n    print(elem[0].shape)\n    print(elem)\n    count += 1\n\ndel dataset\n"

In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in1, out1, in2, out2, st1=1, st2=1, dropout=0.2):
        super(ConvBlock, self).__init__()
        # nn.Conv2D(in_channels, out_channels, kernel_size=(,), stride)
        self.conv1 = nn.Conv2d(in1, out1, kernel_size=(2,2), stride=st1)
        self.conv2 = nn.Conv2d(in2, out2, kernel_size=(2,2), stride=st2)
        self.relu  = nn.ReLU()
        self.pool  = nn.MaxPool2d(kernel_size=(2,2))
        self.bn    = nn.BatchNorm2d(out2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x)
        return x

In [46]:
class ConvBlock3(nn.Module):
    def __init__(self, in1, out1, in2, out2, st1=1, st2=1, dropout=0.2):
        super(ConvBlock3, self).__init__()
        # nn.Conv2D(in_channels, out_channels, kernel_size=(,), stride)
        self.conv1 = nn.Conv2d(in1, out1, kernel_size=(2,3), stride=st1)
        self.conv2 = nn.Conv2d(in2, out2, kernel_size=(2,3), stride=st2)
        self.relu  = nn.ReLU()
        self.pool  = nn.MaxPool2d(kernel_size=(2,2))
        self.bn    = nn.BatchNorm2d(out2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pool(x)
        return x

In [ ]:
class ResBlock1(nn.Module):
    def __init__(self, in1, out1, in2, out2, st1=1, st2=1, dropout=0.2):
        super(ResBlock1, self).__init__()
        # nn.Conv2D(in_channels, out_channels, kernel_size=(,), stride)
        self.conv1 = nn.Conv2d(in1, out1, kernel_size=(2,3), stride=st1)
        self.conv2 = nn.Conv2d(in2, out2, kernel_size=(2,3), stride=st2)
        self.relu  = nn.ReLU()
        self.pool  = nn.MaxPool2d(kernel_size=(2,2))
        self.bn1   = nn.BatchNorm2d(out2)
        self.bn2   = nn.BatchNorm2d(out2)
    
    def forward(self, x):
        residual = x.clone()
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        
        if self.downsample is not None:
            residual = self.downsample(x)
            
        x += residual
        
        x = self.pool(x)
        return x

In [47]:
class CNNEncoder(nn.Module):
    def __init__(self, config):
        super(CNNEncoder, self).__init__()
        n_ch = config['n_channels']
        i_ch = config['input_channels']
        f1_len = config['fc_intermediate_len']
        f2_len = config['output_channels'] # len(y)
        self.conv_block_1 = ConvBlock3(in1=i_ch, out1=n_ch, in2=n_ch, out2=n_ch, st1=2)
        self.conv_block_2 = ConvBlock( in1=n_ch, out1=n_ch, in2=n_ch, out2=n_ch)
        self.conv_block_3 = ConvBlock3(in1=n_ch, out1=n_ch, in2=n_ch, out2=n_ch)
        self.conv_block_4 = ConvBlock( in1=n_ch, out1=n_ch, in2=n_ch, out2=n_ch)
        self.conv_block_5 = ConvBlock( in1=n_ch, out1=n_ch, in2=n_ch, out2=n_ch)
        self.fc1 = nn.Linear(n_ch * 12, f1_len) # FIXME dimensions # 612
        self.fc2 = nn.Linear(f1_len, f2_len)
        self.relu = nn.ReLU()
        self.dropout=nn.Dropout(config['dropout'])
    
    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.conv_block_4(x)
        print('\n', x.shape, '\n', sep='')
        x = self.conv_block_5(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
device

device(type='cuda')

In [6]:
def calc_loss(cnn, data_loader, loss_type='mse'):
    if loss_type =='mse': 
        Loss = nn.MSELoss()
    elif loss_type == 'mae':
        Loss = nn.L1Loss()
    else:
        return 'ERROR'

    loss = 0.0
    cnn.eval()
    with torch.no_grad():
        for x_batch, y_batch in data_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            predictions = cnn(x_batch)
            loss += Loss(predictions, y_batch).item()
    return loss

In [7]:
def train_epoch(data_loader, cnn, optimizer, criterion):
    running_loss = 0.0
    for x_batch, y_batch in tqdm(data_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        predictions = cnn(x_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    print('running_loss after epoch is', running_loss)

In [14]:
def train_model(dataset_train, dataset_test, cnn, n_epochs, batch_size):
    data_train_loader = torch.utils.data.DataLoader(dataset_train, batch_size,
                                                   num_workers=10, persistent_workers=False,pin_memory=True)
    data_test_loader  = torch.utils.data.DataLoader(dataset_test, batch_size, num_workers=2,pin_memory=False)

    criterion = nn.MSELoss()
    #criterion = nn.L1Loss()
    #criterion = nn.SmoothL1Loss()
    optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)
    for i in range(n_epochs):
        print('Starting epoch {}'.format(i))
        cnn.train()
        train_epoch(data_train_loader, cnn, optimizer, criterion)
        #train_loss = 'dummy'#calc_loss(cnn, data_train_loader)
        test_loss = calc_loss(cnn, data_test_loader, 'mae')
        #print('Train loss:\t', train_loss)
        print('Test loss (MAE):\t', test_loss)

In [9]:
"""
dataset_train = PretrainRegressionDataset('data/pretrain2/data_train.csv',
                                          'data/pretrain2/atom_counts_train.npy',
                                           x_size=cfg['x_size'], y_size=cfg['y_size'])
dataset_test = PretrainRegressionDataset('data/pretrain2/data_test.csv',
                                          'data/pretrain2/atom_counts_test.npy',
                                          x_size=cfg['x_size'], y_size=cfg['y_size'])


"""

dataset_train = PretrainRegressionDataset('data/pretrain2/data_train_small.csv',
                                          'data/pretrain2/atom_counts_train_small.npy',
                                           x_size=cfg['x_size'], y_size=cfg['y_size'])
dataset_test = PretrainRegressionDataset('data/pretrain2/data_test_small.csv',
                                          'data/pretrain2/atom_counts_test_small.npy',
                                          x_size=cfg['x_size'], y_size=cfg['y_size'])


In [48]:
cnn = CNNEncoder(cfg)

In [20]:
gc.collect()
torch.cuda.empty_cache()

In [60]:
del cnn

In [10]:
from ResNet import some_model, ResNet50

In [11]:
resnet = some_model(1, 1)

In [13]:
"""
from torchvision.models import resnet34
cnn1 = resnet34(pretrained=False)
"""


'\nfrom torchvision.models import resnet34\ncnn1 = resnet34(pretrained=False)\n'

In [12]:
resnet.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, m

In [15]:
%%time
train_model(dataset_train, dataset_test, resnet, n_epochs=2, batch_size=20)

  0%|          | 0/400 [00:00<?, ?it/s]

Starting epoch 0


100%|██████████| 400/400 [02:54<00:00,  2.30it/s]

running_loss after epoch is 6300.271611690521



  0%|          | 0/400 [00:00<?, ?it/s]

Test loss (MAE):	 783.1421613693237
Starting epoch 1


100%|██████████| 400/400 [02:54<00:00,  2.29it/s]

running_loss after epoch is 4503.52955698967


Test loss (MAE):	 361.4475269317627
CPU times: user 4min 45s, sys: 1min 34s, total: 6min 19s
Wall time: 6min 17s


In [16]:
preds = []
trues = []
resnet.eval()
for index, (_, __) in enumerate(dataset_test):
    if index > 5:
        break
    with torch.no_grad():
        pred = resnet(_.unsqueeze(0).to(device)).item()
    preds.append(pred)
    trues.append(__.item())
        
print(*preds)
print(*trues)

13.347800254821777 22.187238693237305 26.297483444213867 12.539535522460938 26.771360397338867 21.815122604370117
13.0 19.0 21.0 11.0 21.0 16.0


In [17]:
torch.save(resnet.state_dict(), 'data/resnet_v1.model')